In [31]:
# -*- coding: utf-8 -*-
import xlrd 
import numpy
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import LeaveOneOut
import math
import numpy.linalg as LA
from sklearn.cluster import KMeans
import random
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection,metrics
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
random.sample
import pandas as pd
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from collections import Counter
startTime = time.time()

# run

In [32]:
def load_data(directory):
    GSSM = np.loadtxt(directory + '\GSSM_.txt',dtype=np.float32)
    PESSM = np.loadtxt(directory + '\PSSM.txt',dtype=np.float32,delimiter='\t')

    IPE = pd.DataFrame(PESSM).reset_index()
    IG = pd.DataFrame(GSSM).reset_index()
    IPE.rename(columns = {'index':'id'}, inplace = True)
    IG.rename(columns = {'index':'id'}, inplace = True)
    IPE['id'] = IPE['id']
    IG['id'] = IG['id']
    
    return IPE, IG

In [33]:
def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/all_gpe_pairs.csv')
    known_associations = all_associations.loc[all_associations['label'] == 1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)

    sample_df = known_associations.append(random_negative)
    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

In [34]:
def obtain_data(directory, isbalance):

    IPE, IG = load_data(directory)

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/all_gene_peco_pairs.csv')

    gene_ids = list(set(dtp['gene_idx']))
    peco_ids = list(set(dtp['peco_idx']))
    random.shuffle(gene_ids)
    random.shuffle(peco_ids)
    print('# gene = {} | peco = {}'.format(len(gene_ids), len(peco_ids)))

    gene_test_num = int(len(gene_ids) / 5)
    peco_test_num = int(len(peco_ids) / 5)
    print('# Test: gene = {} | peco = {}'.format(gene_test_num, peco_test_num))    
    
    samples = pd.merge(pd.merge(dtp, IPE, left_on = 'peco_idx', right_on = 'id'), IG, left_on = 'gene_idx', right_on = 'id')
    samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    
    return dtp, gene_ids, peco_ids, gene_test_num, peco_test_num, samples


In [35]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if tp+fn == 0:
        recall = 0
    else:
        recall = tp / (tp+fn)
    
    if tp+fp == 0:
        precision = 0
    else:
        precision = tp / (tp+fp)
    
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr)


In [36]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])):
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['gene_idx', 'peco_idx']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['gene_idx', 'peco_idx']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all


def generate_task_Tg_Tpe_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        
    return train_index_all, test_index_all, train_id_all, test_id_all


In [37]:
def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [38]:
def run(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('----------------------- Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']
        x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        x_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
        GBDT=GradientBoostingClassifier(n_estimators = 12, max_depth = 5, min_samples_leaf = 13)
        GBDT.fit(x_train, y_train)
        OHE = OneHotEncoder()
        OHE.fit(GBDT.apply(x_train)[:, :, 0])
        LR = LogisticRegression(n_jobs = -1)
        LR.fit(OHE.transform(GBDT.apply(x_train)[:, :, 0]),y_train)
        
        y_train_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_train)[:, :, 0]))[:,1]
        y_test_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_test)[:, :, 0]))[:,1]

        y_train_pred, y_test_pred = transfer(y_train_prob), transfer(y_test_prob)

        print(len(y_train_pred), len(y_train))
        performances_train = performances(y_train, y_train_pred, y_train_prob)
        performances_test = performances(y_test, y_test_pred, y_test_prob)

        fold += 1
    
    return y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test


# Tp

In [40]:
directory = '../../data'
for isbalance in [True]:
    dtp, gene_ids, peco_ids, gene_test_num, peco_test_num, samples = obtain_data(directory,isbalance)  
    for task in ['Tp', 'Tg', 'Tpe']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tg':
            item = 'gene_idx'
            ids = gene_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, samples)

        elif task == 'Tpe':
            item = 'peco_idx'
            ids = peco_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, samples)

        y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test = run(train_index_all, test_index_all, samples)



5it [00:00, 161.12it/s]

# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 37692 | Test = 9424
-------Fold  1
# Pairs: Train = 37693 | Test = 9423
-------Fold  2
# Pairs: Train = 37693 | Test = 9423
-------Fold  3
# Pairs: Train = 37693 | Test = 9423
-------Fold  4
# Pairs: Train = 37693 | Test = 9423
----------------------- Fold =  0



<ipython-input-37-162246ac2ecc>:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred.reshape(-1)]


37692 37692
tn = 15778, fp = 3112, fn = 1988, tp = 16814
y_pred: 0 = 17766 | 1 = 19926
y_true: 0 = 18890 | 1 = 18802
acc=0.8647|precision=0.8438|recall=0.8943|f1=0.8683|auc=0.9466|aupr=0.9481
tn = 3895, fp = 773, fn = 514, tp = 4242
y_pred: 0 = 4409 | 1 = 5015
y_true: 0 = 4668 | 1 = 4756
acc=0.8634|precision=0.8459|recall=0.8919|f1=0.8683|auc=0.9416|aupr=0.9437
----------------------- Fold =  1


<ipython-input-37-162246ac2ecc>:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred.reshape(-1)]


...

<ipython-input-37-162246ac2ecc>:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred.reshape(-1)]


37261 37261
tn = 17015, fp = 2909, fn = 1759, tp = 15578
y_pred: 0 = 18774 | 1 = 18487
y_true: 0 = 19924 | 1 = 17337
acc=0.8747|precision=0.8426|recall=0.8985|f1=0.8697|auc=0.9546|aupr=0.9523
tn = 3633, fp = 1, fn = 6214, tp = 7
y_pred: 0 = 9847 | 1 = 8
y_true: 0 = 3634 | 1 = 6221
acc=0.3694|precision=0.8750|recall=0.0011|f1=0.0022|auc=0.7723|aupr=0.8434
----------------------- Fold =  3


<ipython-input-37-162246ac2ecc>:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred.reshape(-1)]


32149 32149
tn = 17892, fp = 2473, fn = 2646, tp = 9138
y_pred: 0 = 20538 | 1 = 11611
y_true: 0 = 20365 | 1 = 11784
acc=0.8408|precision=0.7870|recall=0.7755|f1=0.7812|auc=0.9191|aupr=0.8679
tn = 1222, fp = 1971, fn = 10562, tp = 1212
y_pred: 0 = 11784 | 1 = 3183
y_true: 0 = 3193 | 1 = 11774
acc=0.1626|precision=0.3808|recall=0.1029|f1=0.1621|auc=0.0843|aupr=0.5260
----------------------- Fold =  4
37662 37662
tn = 11987, fp = 3170, fn = 2217, tp = 20288
y_pred: 0 = 14204 | 1 = 23458
y_true: 0 = 15157 | 1 = 22505
acc=0.8570|precision=0.8649|recall=0.9015|f1=0.8828|auc=0.9365|aupr=0.9564
tn = 6583, fp = 1818, fn = 197, tp = 856
y_pred: 0 = 6780 | 1 = 2674
y_true: 0 = 8401 | 1 = 1053
acc=0.7869|precision=0.3201|recall=0.8129|f1=0.4594|auc=0.8204|aupr=0.4335


<ipython-input-37-162246ac2ecc>:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred.reshape(-1)]
